In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3tand1k/DATASET/1.2.2026.listens
/kaggle/input/3tand1k/DATASET/8.1.2026.listens
/kaggle/input/3tand1k/DATASET/17.1.2026.listens
/kaggle/input/3tand1k/DATASET/10.1.2026.listens
/kaggle/input/3tand1k/DATASET/24.1.2026.listens
/kaggle/input/3tand1k/DATASET/29.1.2026.listens
/kaggle/input/3tand1k/DATASET/13.1.2026.listens
/kaggle/input/3tand1k/DATASET/19.1.2026.listens
/kaggle/input/3tand1k/DATASET/9.1.2026.listens
/kaggle/input/3tand1k/DATASET/2.2.2026.listens
/kaggle/input/3tand1k/DATASET/30.1.2026.listens
/kaggle/input/3tand1k/DATASET/27.1.2026.listens
/kaggle/input/3tand1k/DATASET/15.1.2026.listens
/kaggle/input/3tand1k/DATASET/12.1.2026.listens
/kaggle/input/3tand1k/DATASET/18.1.2026.listens
/kaggle/input/3tand1k/DATASET/4.2.2026.listens
/kaggle/input/3tand1k/DATASET/5.2.2026.listens
/kaggle/input/3tand1k/DATASET/23.1.2026.listens
/kaggle/input/3tand1k/DATASET/11.1.2026.listens
/kaggle/input/3tand1k/DATASET/readdata.py
/kaggle/input/3tand1k/DATASET/20.1.2026.listens
/kag

In [2]:
import os
import json
import pandas as pd
from datetime import datetime


In [3]:
INPUT_DIR  = "/kaggle/input/3tand1k/DATASET"
OUTPUT_DIR = "/kaggle/working/cleaned_data"

os.makedirs(OUTPUT_DIR, exist_ok=True)


In [4]:
date_list = [
    "22.1.2026",
    "23.1.2026",
    "24.1.2026",
    "25.1.2026",
    "26.1.2026",
    "27.1.2026",
    "28.1.2026",
]


In [5]:
for date_str in date_list:
    input_path  = os.path.join(INPUT_DIR, f"{date_str}.listens")
    output_path = os.path.join(OUTPUT_DIR, f"clean_{date_str}.csv")

    if not os.path.exists(input_path):
        print(f"File không tồn tại: {date_str}")
        continue

    rows = []

    # 1. READ + PARSE JSON
    with open(input_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                record = json.loads(line)

                rows.append({
                    "user_id": record.get("user_id"),
                    "timestamp": record.get("timestamp"),
                    "track_name": record.get("track_metadata", {}).get("track_name"),
                    "artist_name": record.get("track_metadata", {}).get("artist_name"),
                    "release_name": record.get("track_metadata", {}).get("release_name"),
                    "recording_msid": record.get("recording_msid"),
                })

            except json.JSONDecodeError:
                continue

    df = pd.DataFrame(rows)

    # 2. XỬ LÝ THIẾU DỮ LIỆU
    df = df.dropna(subset=[
        "user_id",
        "timestamp",
        "track_name",
        "artist_name",
        "recording_msid"
    ])

    # 3. XỬ LÝ THỜI GIAN

    df["datetime"] = pd.to_datetime(
        df["timestamp"],
        unit="s",
        errors="coerce"
    )

    df = df.dropna(subset=["datetime"])

    df["date"] = df["datetime"].dt.date
    df["hour"] = df["datetime"].dt.hour

    # 4. LÀM SẠCH TEXT

    text_cols = ["track_name", "artist_name", "release_name"]

    for col in text_cols:
        df[col] = (
            df[col]
            .astype(str)
            .str.lower()
            .str.strip()
        )


    # 5. LOẠI BỎ TRÙNG LẶP

    df = df.drop_duplicates(
        subset=["user_id", "recording_msid", "timestamp"]
    )

    # 6. LỌC DATA KHÔNG HỢP LỆ
    df = df[df["timestamp"] > 0]
    df = df[df["track_name"] != ""]
    df = df[df["artist_name"] != ""]

    # 7. RESET INDEX
    df = df.reset_index(drop=True)

    # 8. LƯU FILE DATA SẠCH
    df.to_csv(output_path, index=False)

    print(f"{date_str} | rows sau clean: {len(df)}")

22.1.2026 | rows sau clean: 5685007
23.1.2026 | rows sau clean: 2617071
24.1.2026 | rows sau clean: 2106833
25.1.2026 | rows sau clean: 3189072
26.1.2026 | rows sau clean: 2445031
27.1.2026 | rows sau clean: 3775332
28.1.2026 | rows sau clean: 3729747
